In [2]:
!python -V

Python 3.9.12


#### Import important libraries

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

# Assuming df is your DataFrame
pd.set_option('display.max_columns', None)

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

#### Download Yellow NYC Taxi data for Jan and Feb 2023

In [5]:
# For January 2023
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [6]:
# For Feburary 2023
url2 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet'

#### Q1. Downloading the data

In [7]:
# No. of columns in the January 2023 dataframe
len(df.columns)

19

#### Q2. Computing duration

In [8]:
type(df.tpep_pickup_datetime[0])

pandas._libs.tslibs.timestamps.Timestamp

In [9]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime

In [10]:
df['duration'] = df.duration.apply(lambda td: td.total_seconds()/60)

In [11]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN,13.983333
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN,19.450000
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN,24.516667
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN,13.000000


In [12]:
# standard deviation of the trips duration in January 2023
df.duration.std()

42.594351241920904

#### Q3. Dropping outliers

In [13]:
df.duration.describe(percentiles=[0.75, 0.95]).apply(lambda x: f'{x:,.0f}')

count    3,066,766
mean            16
std             43
min            -29
50%             12
75%             18
95%             36
max         10,029
Name: duration, dtype: object

In [14]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [15]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN,13.983333
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN,19.450000
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN,24.516667
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN,13.000000


In [16]:
# What fraction of the records left after you dropped the outliers?
(3009173 /3066766 )*100

98.1220282212598

#### Q4. One-hot encoding

In [17]:
categorical = ['PULocationID','DOLocationID']

In [18]:
df[categorical] = df[categorical].astype(str)

/tmp/ipykernel_62917/1767203464.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)


In [19]:
# COnvert datafromae into dictionary, i.e each row of dataframe is converted into a dictionary.
train_dicts = df[categorical].to_dict(orient = 'records')

In [20]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [21]:
# What's the dimensionality of this matrix (number of columns)?
X_train.shape[1]

515

#### Q5. Training a model

In [22]:
target = 'duration'
Y_train = df[target]

In [23]:
#Linear Regression
lr = LinearRegression()
lr.fit(X_train, Y_train)

LinearRegression()

In [24]:
Y_preds = lr.predict(X_train)

In [25]:
# What's the RMSE on train?
mse = mean_squared_error(Y_train, Y_preds)

rmse = np.sqrt(mse)
rmse

7.6492610279057605

#### Q6. Evaluating the model for validation dataset Feb 2023


In [26]:
val_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [27]:
# No. of columns in the January 2023 dataframe
len(val_df.columns)

19

In [28]:
val_df['duration'] = val_df.tpep_dropoff_datetime - val_df.tpep_pickup_datetime
val_df['duration'] = val_df.duration.apply(lambda td: td.total_seconds()/60)

In [29]:
val_df = val_df[(val_df.duration >= 1) & (val_df.duration <= 60)]

In [31]:
categorical = ['PULocationID','DOLocationID']
val_df[categorical] = val_df[categorical].astype(str)

# COnvert datafromae into dictionary, i.e each row of dataframe is converted into a dictionary.
val_dicts = val_df[categorical].to_dict(orient = 'records')
dv = DictVectorizer()
X_val = dv.fit_transform(val_dicts)

In [34]:
X_val.shape[1]

514

In [35]:
target = 'duration'
Y_val = val_df[target]

In [36]:
Y_preds = lr.predict(X_val)

ValueError: X has 514 features, but LinearRegression is expecting 515 features as input.

In [ ]:
# What's the RMSE on train?
mse = mean_squared_error(Y_val, Y_preds)

rmse = np.sqrt(mse)
rmse

In [1]:
ls

data/  duration-prediction.ipynb  homework.ipynb  models/
